In [1]:
import requests
import pandas as pd
import json
import numpy as np

# Get Weather Data


Geo coords for Dhaka, Chittagong, and Patuakhali


In [2]:
# tuple as (lat, lon)
dhk_coords = (23.7104, 90.40744)
cht_coords = (22.3384, 91.83168)
pat_coords = (22.36833, 90.3458)

In [3]:
def get_city_weather(coords):
    weather_api_url = f"https://api.open-meteo.com/v1/forecast?latitude={coords[0]}&longitude={coords[1]}&daily=relative_humidity_2m_mean,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum,sunshine_duration&forecast_days=7"
    response = requests.get(weather_api_url)

    try:
        response.raise_for_status()
        data = response.json()

        return data
    except requests.exceptions.RequestException as e:
        print(e)

In [4]:
dhaka_data = get_city_weather(dhk_coords)
dhaka_data

{'latitude': 23.75,
 'longitude': 90.375,
 'generationtime_ms': 0.08153915405273438,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 27.0,
 'daily_units': {'time': 'iso8601',
  'relative_humidity_2m_mean': '%',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'temperature_2m_mean': '°C',
  'rain_sum': 'mm',
  'sunshine_duration': 's'},
 'daily': {'time': ['2025-08-29',
   '2025-08-30',
   '2025-08-31',
   '2025-09-01',
   '2025-09-02',
   '2025-09-03',
   '2025-09-04'],
  'relative_humidity_2m_mean': [87, 84, 82, 78, 91, 78, 79],
  'temperature_2m_max': [31.3, 32.0, 32.7, 34.4, 28.0, 33.5, 33.1],
  'temperature_2m_min': [26.0, 25.9, 26.0, 26.5, 25.6, 25.9, 26.3],
  'temperature_2m_mean': [27.9, 28.5, 29.2, 30.1, 26.9, 29.4, 29.2],
  'rain_sum': [0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0],
  'sunshine_duration': [26737.15,
   39087.01,
   40759.1,
   37625.68,
   0.0,
   41139.4,
   39697.26]}}

In [5]:
city_weather_df = pd.DataFrame(
    columns=[
        "Date",  # Remove Date before passed to model
        "Rainfall",
        "Sunshine",
        "Humidity",
        "Temp_mean",
        "Temp_max",
        "Temp_min",
        "Year",
        "Month",
        "loadshed_prev",
        "generation_prev",
    ]
)
city_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev


In [6]:
dhaka_data["daily"]

{'time': ['2025-08-29',
  '2025-08-30',
  '2025-08-31',
  '2025-09-01',
  '2025-09-02',
  '2025-09-03',
  '2025-09-04'],
 'relative_humidity_2m_mean': [87, 84, 82, 78, 91, 78, 79],
 'temperature_2m_max': [31.3, 32.0, 32.7, 34.4, 28.0, 33.5, 33.1],
 'temperature_2m_min': [26.0, 25.9, 26.0, 26.5, 25.6, 25.9, 26.3],
 'temperature_2m_mean': [27.9, 28.5, 29.2, 30.1, 26.9, 29.4, 29.2],
 'rain_sum': [0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0],
 'sunshine_duration': [26737.15,
  39087.01,
  40759.1,
  37625.68,
  0.0,
  41139.4,
  39697.26]}

In [7]:
daily_data = dhaka_data["daily"]

city_weather_df["Date"] = daily_data["time"]
city_weather_df["Date"] = pd.to_datetime(city_weather_df["Date"], format="%Y-%m-%d")
city_weather_df["Year"] = city_weather_df["Date"].dt.year
city_weather_df["Month"] = city_weather_df["Date"].dt.month

city_weather_df["Rainfall"] = daily_data["rain_sum"]
city_weather_df["Sunshine"] = daily_data["sunshine_duration"]
city_weather_df["Sunshine"] = city_weather_df["Sunshine"] / (60 * 60)
city_weather_df["Humidity"] = daily_data["relative_humidity_2m_mean"]

city_weather_df["Temp_mean"] = daily_data["temperature_2m_mean"]
city_weather_df["Temp_max"] = daily_data["temperature_2m_max"]
city_weather_df["Temp_min"] = daily_data["temperature_2m_min"]

city_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.0,7.426986,87,27.9,31.3,26.0,2025,8,NaN,NaN
1,2025-08-30,0.0,10.857503,84,28.5,32.0,25.9,2025,8,NaN,NaN
2,2025-08-31,0.0,11.321972,82,29.2,32.7,26.0,2025,8,NaN,NaN
3,2025-09-01,0.0,10.451578,78,30.1,34.4,26.5,2025,9,NaN,NaN
4,2025-09-02,0.3,0.000000,91,26.9,28.0,25.6,2025,9,NaN,NaN
5,2025-09-03,0.0,11.427611,78,29.4,33.5,25.9,2025,9,NaN,NaN
6,2025-09-04,0.0,11.027017,79,29.2,33.1,26.3,2025,9,NaN,NaN


In [8]:
def get_city_weather_df(daily_data):
    city_df = pd.DataFrame(
        columns=[
            "Date",  # Remove Date before passed to model
            "Rainfall",
            "Sunshine",
            "Humidity",
            "Temp_mean",
            "Temp_max",
            "Temp_min",
            "Year",
            "Month",
            "loadshed_prev",
            "generation_prev",
        ]
    )

    city_df["Date"] = daily_data["time"]
    city_df["Date"] = pd.to_datetime(city_df["Date"], format="%Y-%m-%d")
    city_df["Year"] = city_df["Date"].dt.year
    city_df["Month"] = city_df["Date"].dt.month

    city_df["Rainfall"] = daily_data["rain_sum"]
    city_df["Sunshine"] = daily_data["sunshine_duration"]
    # Convert seconds to hours
    city_df["Sunshine"] = city_df["Sunshine"] / (60 * 60)
    city_df["Humidity"] = daily_data["relative_humidity_2m_mean"]

    city_df["Temp_mean"] = daily_data["temperature_2m_mean"]
    city_df["Temp_max"] = daily_data["temperature_2m_max"]
    city_df["Temp_min"] = daily_data["temperature_2m_min"]

    return city_df

## Get weather data of all locations


In [9]:
dhk_data = get_city_weather(dhk_coords)
dhk_weather_df = get_city_weather_df(dhk_data["daily"])
dhk_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.0,7.426986,87,27.9,31.3,26.0,2025,8,NaN,NaN
1,2025-08-30,0.0,10.857503,84,28.5,32.0,25.9,2025,8,NaN,NaN
2,2025-08-31,0.0,11.321972,82,29.2,32.7,26.0,2025,8,NaN,NaN
3,2025-09-01,0.0,10.451578,78,30.1,34.4,26.5,2025,9,NaN,NaN
4,2025-09-02,0.3,0.000000,91,26.9,28.0,25.6,2025,9,NaN,NaN
5,2025-09-03,0.0,11.427611,78,29.4,33.5,25.9,2025,9,NaN,NaN
6,2025-09-04,0.0,11.027017,79,29.2,33.1,26.3,2025,9,NaN,NaN


In [10]:
cht_data = get_city_weather(cht_coords)
cht_weather_df = get_city_weather_df(cht_data["daily"])
cht_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.0,9.615403,86,27.9,28.3,27.2,2025,8,NaN,NaN
1,2025-08-30,0.0,8.101661,85,28.0,28.6,27.1,2025,8,NaN,NaN
2,2025-08-31,0.0,10.996172,84,28.4,29.4,27.1,2025,8,NaN,NaN
3,2025-09-01,34.6,11.319458,83,28.8,30.7,26.3,2025,9,NaN,NaN
4,2025-09-02,38.6,3.785947,85,28.0,29.0,25.7,2025,9,NaN,NaN
5,2025-09-03,0.0,11.548733,82,29.0,30.1,27.6,2025,9,NaN,NaN
6,2025-09-04,0.0,11.582036,81,28.6,30.0,27.6,2025,9,NaN,NaN


In [11]:
pat_data = get_city_weather(pat_coords)
pat_weather_df = get_city_weather_df(pat_data["daily"])
pat_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.0,6.051331,88,27.8,30.0,26.2,2025,8,NaN,NaN
1,2025-08-30,0.0,6.381333,87,28.2,31.6,26.3,2025,8,NaN,NaN
2,2025-08-31,0.0,11.499358,85,28.7,32.0,26.3,2025,8,NaN,NaN
3,2025-09-01,0.6,11.334900,79,29.7,33.9,26.4,2025,9,NaN,NaN
4,2025-09-02,23.1,1.142328,91,26.9,28.4,26.1,2025,9,NaN,NaN
5,2025-09-03,0.0,10.672853,85,28.4,31.0,26.2,2025,9,NaN,NaN
6,2025-09-04,0.0,9.962567,85,28.2,30.5,26.4,2025,9,NaN,NaN


In [12]:
all_weather_df = pd.concat([dhk_weather_df, cht_weather_df, pat_weather_df])

all_weather_df = (
    all_weather_df.groupby("Date")
    .agg(
        {
            "Rainfall": "mean",
            "Sunshine": "mean",
            "Humidity": "mean",
            "Temp_mean": "mean",
            "Temp_max": "max",
            "Temp_min": "min",
            "Year": "first",
            "Month": "first",
            "loadshed_prev": "first",
            "generation_prev": "first",
        }
    )
    .reset_index()
)

all_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.000000,7.697906,87.000000,27.866667,31.3,26.0,2025,8,None,None
1,2025-08-30,0.000000,8.446832,85.333333,28.233333,32.0,25.9,2025,8,None,None
2,2025-08-31,0.000000,11.272501,83.666667,28.766667,32.7,26.0,2025,8,None,None
3,2025-09-01,11.733333,11.035312,80.000000,29.533333,34.4,26.3,2025,9,None,None
4,2025-09-02,20.666667,1.642758,89.000000,27.266667,29.0,25.6,2025,9,None,None
5,2025-09-03,0.000000,11.216399,81.666667,28.933333,33.5,25.9,2025,9,None,None
6,2025-09-04,0.000000,10.857206,81.666667,28.666667,33.1,26.3,2025,9,None,None


# Get Previous Day's Electricity Data


In [13]:
from datetime import datetime, timedelta

yesterday_date = datetime.today() - timedelta(days=1)

yesterday_date = yesterday_date.strftime("%d-%m-%Y")
yesterday_date

'28-08-2025'

In [14]:
import io

pgcb_url = f"https://erp.pgcb.gov.bd/w/generations/view_generations?page={1}"

try:
    pgcb_res = requests.get(pgcb_url, verify=False)
    pgcb_res.raise_for_status()
    html = pgcb_res.text

    tables = pd.read_html(io.StringIO(html))

    electricity_df = tables[0]

    electricity_df.columns = electricity_df.columns.droplevel(0)
    electricity_df = electricity_df[
        ["Date", "Generation(MW)", "Demand(MW)", "Loadshed"]
    ].copy()

    electricity_df = electricity_df[electricity_df["Date"] == yesterday_date]

    electricity_df.rename(
        columns={
            "Generation(MW)": "Generation",
            "Demand(MW)": "Demand",
        },
        inplace=True,
    )

    # Convert units from MW to GW
    electricity_df[["Generation", "Demand", "Loadshed"]] = (
        electricity_df[["Generation", "Demand", "Loadshed"]] / 1000
    )
except requests.exceptions.RequestException as e:
    print(e)

/home/mashrur/anaconda3/envs/my_env/lib/python3.12/site-packages/urllib3/connectionpool.py:2303: InsecureRequestWarning: Unverified HTTPS request is being made to host 'erp.pgcb.gov.bd'. Adding certificate verification is strongly advised. See: https://urllib3future.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
electricity_df

,Date,Generation,Demand,Loadshed
10,28-08-2025,15.150,15.150,0.000
11,28-08-2025,15.241,15.241,0.000
12,28-08-2025,15.117,15.117,0.000
13,28-08-2025,15.243,15.243,0.000
14,28-08-2025,15.278,15.278,0.000
15,28-08-2025,15.505,15.505,0.000
16,28-08-2025,14.992,14.992,0.000
17,28-08-2025,13.060,13.060,0.000
18,28-08-2025,13.522,13.522,0.000
19,28-08-2025,13.853,13.853,0.000


In [16]:
input_df = all_weather_df.copy()

# input_df.loc[0, "generation_prev"] = electricity_df["Generation"].sum()
# input_df.loc[0, "loadshed_prev"] = electricity_df["Loadshed"].sum()
input_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-29,0.000000,7.697906,87.000000,27.866667,31.3,26.0,2025,8,None,None
1,2025-08-30,0.000000,8.446832,85.333333,28.233333,32.0,25.9,2025,8,None,None
2,2025-08-31,0.000000,11.272501,83.666667,28.766667,32.7,26.0,2025,8,None,None
3,2025-09-01,11.733333,11.035312,80.000000,29.533333,34.4,26.3,2025,9,None,None
4,2025-09-02,20.666667,1.642758,89.000000,27.266667,29.0,25.6,2025,9,None,None
5,2025-09-03,0.000000,11.216399,81.666667,28.933333,33.5,25.9,2025,9,None,None
6,2025-09-04,0.000000,10.857206,81.666667,28.666667,33.1,26.3,2025,9,None,None


In [17]:
input_df.dtypes

Date               datetime64[ns]
Rainfall                  float64
Sunshine                  float64
Humidity                  float64
Temp_mean                 float64
Temp_max                  float64
Temp_min                  float64
Year                        int32
Month                       int32
loadshed_prev              object
generation_prev            object
dtype: object

# Load Model


In [18]:
import lightgbm as lgb

generation_model = lgb.Booster(model_file="models/generation_lgbm_model.txt")
loadshed_model = lgb.Booster(model_file="models/loadshed_lgbm_model.txt")

In [19]:
# Set initial values
generation_previous = electricity_df["Generation"].sum()
loadshed_previous = electricity_df["Loadshed"].sum()

forecasts = []

for idx in range(len(input_df)):
    row = input_df.iloc[idx].to_frame().T.infer_objects()

    row["generation_prev"] = generation_previous
    row["loadshed_prev"] = loadshed_previous

    generation_pred = generation_model.predict(row.drop(columns="Date"))[0]
    generation_previous = generation_pred

    loadshed_pred = loadshed_model.predict(row.drop(columns="Date"))[0]
    loadshed_previous = loadshed_pred

    daily_forecast = {
        "date": row["Date"].astype(str).values[0],
        "prediction": {
            "generation": generation_pred,
            "loadshed": loadshed_pred,
        },
    }
    forecasts.append(daily_forecast)

In [20]:
forecasts

[{'date': '2025-08-29',
  'prediction': {'generation': 188.47441212913688,
   'loadshed': 0.8763538260445272}},
 {'date': '2025-08-30',
  'prediction': {'generation': 191.1213145569318,
   'loadshed': 0.8032772863988772}},
 {'date': '2025-08-31',
  'prediction': {'generation': 195.7298388677536,
   'loadshed': 4.596259441254343}},
 {'date': '2025-09-01',
  'prediction': {'generation': 193.8519678708782,
   'loadshed': 4.030379639969183}},
 {'date': '2025-09-02',
  'prediction': {'generation': 181.71602752742697,
   'loadshed': 0.3804741046823805}},
 {'date': '2025-09-03',
  'prediction': {'generation': 195.56593762570932,
   'loadshed': 4.001081452842083}},
 {'date': '2025-09-04',
  'prediction': {'generation': 195.53897282194075,
   'loadshed': 4.001081452842083}}]